In [7]:
import sklearn
print(sklearn.__version__)
import nltk
import gensim
import string
import numpy as np
import pandas as pd
import joblib

from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer

pd.options.mode.chained_assignment = None

1.2.1


In [8]:
df = pd.read_csv('../row_dataset/Anime_data.csv')
df = df.drop(columns=['Anime_id', 'ScoredBy', 'Popularity', 'Members', 'Episodes', 'Episodes', 'Aired', 'Link'])

In [9]:
df.shape

(17002, 8)

In [10]:
df.head()

,Title,Genre,Synopsis,Type,Producer,Studio,Rating,Source
0,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,Original
1,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,Original
2,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31,Manga
3,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34,Original
4,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04,Manga


In [11]:
df.dtypes

Title        object
Genre        object
Synopsis     object
Type         object
Producer     object
Studio       object
Rating      float64
Source       object
dtype: object

In [12]:
df.columns

Index(['Title', 'Genre', 'Synopsis', 'Type', 'Producer', 'Studio', 'Rating',
       'Source'],
      dtype='object')

In [13]:
df.isna().sum()

Title          0
Genre       2012
Synopsis    1419
Type         634
Producer    9367
Studio      9083
Rating      2577
Source      1927
dtype: int64

In [14]:
# Handling missing values
df = df[(df['Rating'].notnull()) & (df['Source'].notnull()) & (df['Synopsis'].notnull())]

df.loc[df['Producer'].isna(), 'Producer'] = 'unknown'
df.loc[df['Studio'].isna(), 'Studio'] = 'unknown'

In [15]:
# Converting categorical variables into numerical values
df['Type_orig'] = df['Type']
df['Source_orig'] = df['Source']
# df['Producer_orig'] = df['Producer']
# df['Studio_orig'] = df['Studio']

# Factorize the columns
df['Type'] = pd.factorize(df['Type'])[0]
df['Source'] = pd.factorize(df['Source'])[0]
# df['Producer'] = pd.factorize(df['Producer'])[0]
# df['Studio'] = pd.factorize(df['Studio'])[0]

In [16]:
df_type = df[['Type_orig', 'Type']].drop_duplicates()
df_source = df[['Source_orig', 'Source']].drop_duplicates()
# df_producer = df[['Producer_orig', 'Producer']].drop_duplicates()
# df_studio = df[['Studio_orig', 'Studio']].drop_duplicates()

In [17]:
df_type

,Type_orig,Type
0,TV,0
1,Movie,1
26,OVA,2
175,Special,3
486,ONA,4
609,Music,5


In [18]:
df_source

,Source_orig,Source
0,Original,0
2,Manga,1
18,Light novel,2
37,Game,3
45,Visual novel,4
49,4-koma manga,5
69,Novel,6
71,Unknown,7
151,Other,8
284,Picture book,9


In [19]:
df['Genre'].head()

0    ['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...
1    ['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']
2    ['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...
3    ['Action', 'Magic', 'Police', 'Supernatural', ...
4    ['Adventure', 'Fantasy', 'Shounen', 'Supernatu...
Name: Genre, dtype: object

In [20]:
# Replace missing values in the Genre column with an empty list
df['Genre'].fillna(value='[]', inplace=True)

# Converting the genre column to a list of genres
df['Genre'] = df['Genre'].apply(eval)

In [21]:
df['Genre'].head()

0    [Action, Adventure, Comedy, Drama, Sci-Fi, Space]
1              [Action, Space, Drama, Mystery, Sci-Fi]
2    [Action, Sci-Fi, Adventure, Comedy, Drama, Sho...
3    [Action, Magic, Police, Supernatural, Drama, M...
4          [Adventure, Fantasy, Shounen, Supernatural]
Name: Genre, dtype: object

In [22]:
df['Genre'].dtypes

dtype('O')

In [23]:
# Performing one-hot encoding on the genre column
genres = set(g for genres in df['Genre'] for g in genres)
for genre in genres:
    df[genre] = df['Genre'].apply(lambda x: 1 if genre in x else 0)

In [24]:
# Dropping the original genre column
df.drop('Genre', axis=1, inplace=True)

df.head()

,Title,Synopsis,Type,Producer,Studio,Rating,Source,Type_orig,Source_orig,Dementia,...,Sci-Fi,Supernatural,Police,Thriller,Josei,Cars,Music,Vampire,Shoujo Ai,Historical
0,Cowboy Bebop,"In the year 2071, humanity has colonized sever...",0,['Bandai Visual'],['Sunrise'],8.81,0,TV,Original,0,...,1,0,0,0,0,0,0,0,0,0
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...",1,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,0,Movie,Original,0,...,1,0,0,0,0,0,0,0,0,0
2,Trigun,"Vash the Stampede is the man with a $$60,000,0...",0,['Victor Entertainment'],['Madhouse'],8.31,1,TV,Manga,0,...,1,0,0,0,0,0,0,0,0,0
3,Witch Hunter Robin,Witches are individuals with special powers li...,0,['Bandai Visual'],['Sunrise'],7.34,0,TV,Original,0,...,0,1,1,0,0,0,0,0,0,0
4,Bouken Ou Beet,It is the dark century and the people are suff...,0,unknown,['Toei Animation'],7.04,1,TV,Manga,0,...,0,1,0,0,0,0,0,0,0,0


In [25]:
# Define the pre-processing functions
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
  stop_words = set(stopwords.words('english')) 
  return ' '.join([word for word in text.split() if word not in stop_words])

def stem_text(text):
  stemmer = SnowballStemmer('english')
  return ' '.join([stemmer.stem(word) for word in text.split()])

def lemmatize_text(text):
  lemmatizer = WordNetLemmatizer()
  return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])


In [26]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Preprocess the Synopsis of the anime
df['preprocessed_synopsis'] = df['Synopsis'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df['preprocessed_synopsis'] = df['preprocessed_synopsis'].apply(remove_punctuation)
df['preprocessed_synopsis'] = df['preprocessed_synopsis'].apply(remove_stopwords)
df['preprocessed_synopsis'] = df['preprocessed_synopsis'].apply(stem_text)
df['preprocessed_synopsis'] = df['preprocessed_synopsis'].apply(lemmatize_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeanv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jeanv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jeanv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [27]:
# Preprocess the Synopsis of the anime
df['Title'] = df['Title'].astype('str')

df['preprocessed_Title'] = df['Title'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df['preprocessed_Title'] = df['preprocessed_Title'].apply(remove_punctuation)
df['preprocessed_Title'] = df['preprocessed_Title'].apply(remove_stopwords)
df['preprocessed_Title'] = df['preprocessed_Title'].apply(stem_text)
df['preprocessed_Title'] = df['preprocessed_Title'].apply(lemmatize_text)

In [28]:
# Preprocess the Synopsis of the anime
df['Producer'] = df['Producer'].astype('str')

df['preprocessed_Producer'] = df['Producer'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df['preprocessed_Producer'] = df['preprocessed_Producer'].apply(remove_punctuation)
df['preprocessed_Producer'] = df['preprocessed_Producer'].apply(remove_stopwords)
df['preprocessed_Producer'] = df['preprocessed_Producer'].apply(stem_text)
df['preprocessed_Producer'] = df['preprocessed_Producer'].apply(lemmatize_text)

In [29]:
# Preprocess the Synopsis of the anime
df['Studio'] = df['Studio'].astype('str')

df['preprocessed_Studio'] = df['Studio'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df['preprocessed_Studio'] = df['preprocessed_Studio'].apply(remove_punctuation)
df['preprocessed_Studio'] = df['preprocessed_Studio'].apply(remove_stopwords)
df['preprocessed_Studio'] = df['preprocessed_Studio'].apply(stem_text)
df['preprocessed_Studio'] = df['preprocessed_Studio'].apply(lemmatize_text)

In [30]:
# Convert the preprocessed synopsis into a list of tokenized sentences
sentences_synopsis = [synopsis.split() for synopsis in df['preprocessed_synopsis']]
sentences_Title = [title.split() for title in df['preprocessed_Title']]
sentences_Producer = [producer.split() for producer in df['preprocessed_Producer']]
sentences_Studio = [studio.split() for studio in df['preprocessed_Studio']]

In [31]:
# Train a Word2Vec model on the tokenized sentences
w2v_model_synopsis = gensim.models.Word2Vec(sentences=sentences_synopsis, vector_size=100, window=5, min_count=1, workers=-1)
w2v_model_title = gensim.models.Word2Vec(sentences=sentences_Title, vector_size=100, window=5, min_count=1, workers=-1)
w2v_model_producer = gensim.models.Word2Vec(sentences=sentences_Producer, vector_size=100, window=5, min_count=1, workers=-1)
w2v_model_studio = gensim.models.Word2Vec(sentences=sentences_Studio, vector_size=100, window=5, min_count=1, workers=-1)

In [32]:
joblib.dump(w2v_model_synopsis, 'w2v_model_synopsis.pkl')
joblib.dump(w2v_model_title, 'w2v_model_title.pkl')
joblib.dump(w2v_model_producer, 'w2v_model_producer.pkl')
joblib.dump(w2v_model_studio, 'w2v_model_studio.pkl')

['w2v_model_studio.pkl']

In [33]:
# Convert each synopsis into a numeric vector using the trained Word2Vec model
def vectorize_synopsis(synopsis):
    tokens = synopsis.split()
    vectors = [w2v_model_synopsis.wv.get_vector(token) for token in tokens if token in w2v_model_synopsis.wv.key_to_index]
    return sum(vectors) / len(vectors) if vectors else [0] * 100

In [34]:
# Convert each synopsis into a numeric vector using the trained Word2Vec model
def vectorize_title(title):
    tokens = title.split()
    vectors = [w2v_model_title.wv.get_vector(token) for token in tokens if token in w2v_model_title.wv.key_to_index]
    return sum(vectors) / len(vectors) if vectors else [0] * 100

In [35]:
# Convert each synopsis into a numeric vector using the trained Word2Vec model
def vectorize_producer(producer):
    tokens = producer.split()
    vectors = [w2v_model_producer.wv.get_vector(token) for token in tokens if token in w2v_model_producer.wv.key_to_index]
    return sum(vectors) / len(vectors) if vectors else [0] * 100

In [36]:
# Convert each synopsis into a numeric vector using the trained Word2Vec model
def vectorize_studio(studio):
    tokens = studio.split()
    vectors = [w2v_model_studio.wv.get_vector(token) for token in tokens if token in w2v_model_studio.wv.key_to_index]
    return sum(vectors) / len(vectors) if vectors else [0] * 100

In [37]:
df['synopsis_vectors'] = df['preprocessed_synopsis'].apply(vectorize_synopsis)
df['title_vectors'] = df['preprocessed_Title'].apply(vectorize_title)
df['producer_vectors'] = df['preprocessed_Producer'].apply(vectorize_producer)
df['studio_vectors'] = df['preprocessed_Studio'].apply(vectorize_studio)

In [38]:
df

,Title,Synopsis,Type,Producer,Studio,Rating,Source,Type_orig,Source_orig,Dementia,...,Shoujo Ai,Historical,preprocessed_synopsis,preprocessed_Title,preprocessed_Producer,preprocessed_Studio,synopsis_vectors,title_vectors,producer_vectors,studio_vectors
0,Cowboy Bebop,"In the year 2071, humanity has colonized sever...",0,['Bandai Visual'],['Sunrise'],8.81,0,TV,Original,0,...,0,0,year 2071 human colon sever planet moon solar ...,cowboy bebop,bandai visual,sunris,"[-0.0005842843, 0.0010318753, 0.0003277911, 0....","[-0.0035258993, -0.0076524, -0.001318773, 0.00...","[-0.0008193427, 0.00028111576, 0.0035369773, -...","[-0.0087274825, 0.0021301615, -0.0008735442, -..."
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...",1,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,0,Movie,Original,0,...,0,0,anoth day anoth bounty—such life often unlucki...,cowboy bebop tengoku tobira,sunris bandai visual,bone,"[-0.00041281452, 0.0009827461, -1.9511826e-05,...","[0.0014435188, -0.0047937236, -0.0007585628, 0...","[-0.003499575, -0.0029092114, 0.0023581742, -0...","[0.009648528, 0.007324829, 0.0012616563, -0.00..."
2,Trigun,"Vash the Stampede is the man with a $$60,000,0...",0,['Victor Entertainment'],['Madhouse'],8.31,1,TV,Manga,0,...,0,0,vash stamped man 60000000000 bounti head reaso...,trigun,victor entertain,madhous,"[0.0004804233, 0.00017798069, 0.00081434444, 0...","[-0.0012914204, 0.009018698, -0.005590031, 0.0...","[-0.0009417677, -0.0010257291, -0.0076476308, ...","[0.0081322715, -0.004457334, -0.0010683573, 0...."
3,Witch Hunter Robin,Witches are individuals with special powers li...,0,['Bandai Visual'],['Sunrise'],7.34,0,TV,Original,0,...,0,0,witch individu special power like esp telekine...,witch hunter robin,bandai visual,sunris,"[-0.0007236848, 0.0008979221, -0.00080057426, ...","[0.0047848406, 0.00014208879, -0.001255269, -0...","[-0.0008193427, 0.00028111576, 0.0035369773, -...","[-0.0087274825, 0.0021301615, -0.0008735442, -..."
4,Bouken Ou Beet,It is the dark century and the people are suff...,0,unknown,['Toei Animation'],7.04,1,TV,Manga,0,...,0,0,dark centuri peopl suffer rule devil vandel ab...,bouken ou beet,unknown,toei anim,"[-0.00085029885, 0.0021997162, 5.6877518e-05, ...","[-0.0014229012, 0.0012140521, 0.0023713761, -0...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.007879351, 0.0024533845, -0.0009934164, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15015,Ake-Vono,Music video directed by Kouhei Yoshino for the...,5,unknown,unknown,5.52,0,Music,Original,1,...,0,0,music video direct kouhei yoshino song akevono...,akevono,unknown,unknown,"[-0.00030113, 0.0038219758, -0.00013590511, 0....","[0.0075038527, -0.00063257816, -0.0046589146, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ..."
15016,Ab-rah,Music video directed by Kouhei Yoshino for the...,5,unknown,unknown,5.52,0,Music,Original,0,...,0,0,music video direct kouhei yoshino song abrah j...,abrah,unknown,unknown,"[-0.002198061, 0.0022400143, -0.0001623089, -0...","[0.009823671, 0.006244389, 0.004912238, 0.0022...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ..."
15031,Space Shower Man: Legacy for the Future,Space Shower TV Station ID directed by Densuke...,3,unknown,unknown,5.29,0,Special,Original,0,...,0,0,space shower tv station id direct densuke28 so...,space shower man legaci futur,unknown,unknown,"[-0.00095495634, 0.00057553774, 0.0023163885, ...","[0.00048050284, 0.0007103888, 0.0027244545, 0....","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ..."
15042,Mugyutto! Black Clover,SD Black Clover short comedy specials releas...,4,unknown,['DLE'],6.91,1,ONA,Manga,0,...,0,0,sd black clover short comedi special releas ex...,mugyutto black clover,unknown,dle,"[-0.00076304964, -0.001008537, -0.0010246095, ...","[-0.0

In [39]:
# Dropping irrelevant columns
df = df.drop(['Synopsis', 'preprocessed_synopsis', 'Title', 'preprocessed_Title', 'Producer', 'preprocessed_Producer', 'Studio', 'preprocessed_Studio'], axis=1)

In [40]:
df['synopsis_vectors'] = df['synopsis_vectors'].apply(lambda x: np.array(x))
df['title_vectors'] = df['title_vectors'].apply(lambda x: np.array(x))
df['producer_vectors'] = df['producer_vectors'].apply(lambda x: np.array(x))
df['studio_vectors'] = df['studio_vectors'].apply(lambda x: np.array(x))

In [41]:
# Concatenate vectors with the other columns 
X = np.concatenate([df['synopsis_vectors'].values.tolist(),
                    df['title_vectors'].values.tolist(), 
                    df['Type'].values.reshape(-1,1),
                    df['producer_vectors'].values.tolist(), 
                    df['studio_vectors'].values.tolist(), 
                    df['Source'].values.reshape(-1,1),
                    df['Horror'].values.reshape(-1,1),
                    df['Historical'].values.reshape(-1,1),
                    df['Adventure'].values.reshape(-1,1),
                    df['Sci-Fi'].values.reshape(-1,1),
                    df['Shounen'].values.reshape(-1,1),
                    df['Seinen'].values.reshape(-1,1),
                    df['Super Power'].values.reshape(-1,1), 
                    df['Kids'].values.reshape(-1,1),
                    df['Samurai'].values.reshape(-1,1),
                    df['Mystery'].values.reshape(-1,1),
                    df['Police'].values.reshape(-1,1),  
                    df['Yuri'].values.reshape(-1,1),
                    df['Romance'].values.reshape(-1,1),
                    df['Space'].values.reshape(-1,1),
                    df['Game'].values.reshape(-1,1),
                    df['Shoujo'].values.reshape(-1,1),
                    df['Martial Arts'].values.reshape(-1,1),
                    df['Shounen Ai'].values.reshape(-1,1),
                    df['Josei'].values.reshape(-1,1),
                    df['Military'].values.reshape(-1,1),
                    df['Psychological'].values.reshape(-1,1),
                    df['Thriller'].values.reshape(-1,1),
                    df['Parody'].values.reshape(-1,1),
                    df['Music'].values.reshape(-1,1), 
                    df['Mecha'].values.reshape(-1,1),
                    df['Yaoi'].values.reshape(-1,1),
                    df['Supernatural'].values.reshape(-1,1),
                    df['Demons'].values.reshape(-1,1),
                    df['Sports'].values.reshape(-1,1), 
                    df['Action'].values.reshape(-1,1),
                    df['Shoujo Ai'].values.reshape(-1,1),
                    df['Dementia'].values.reshape(-1,1),
                    df['Harem'].values.reshape(-1,1),
                    df['Hentai'].values.reshape(-1,1),
                    df['Cars'].values.reshape(-1,1),
                    df['Slice of Life'].values.reshape(-1,1),
                    df['Drama'].values.reshape(-1,1),
                    df['Fantasy'].values.reshape(-1,1),
                    df['Magic'].values.reshape(-1,1),
                    df['Ecchi'].values.reshape(-1,1),               
                    df['Comedy'].values.reshape(-1,1),],
                    axis=1)

In [42]:
print(X.shape)

(12569, 443)


In [43]:
X

array([[-5.84284309e-04,  1.03187526e-03,  3.27791087e-04, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-4.12814523e-04,  9.82746133e-04, -1.95118264e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.80423303e-04,  1.77980692e-04,  8.14344443e-04, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       ...,
       [-9.54956340e-04,  5.75537735e-04,  2.31638853e-03, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-7.63049640e-04, -1.00853702e-03, -1.02460955e-03, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-1.38610616e-04,  1.25420070e-03,  3.18249135e-04, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [44]:
df

,Type,Rating,Source,Type_orig,Source_orig,Dementia,Slice of Life,Space,Parody,Military,...,Josei,Cars,Music,Vampire,Shoujo Ai,Historical,synopsis_vectors,title_vectors,producer_vectors,studio_vectors
0,0,8.81,0,TV,Original,0,0,1,0,0,...,0,0,0,0,0,0,"[-0.0005842843, 0.0010318753, 0.0003277911, 0....","[-0.0035258993, -0.0076524, -0.001318773, 0.00...","[-0.0008193427, 0.00028111576, 0.0035369773, -...","[-0.0087274825, 0.0021301615, -0.0008735442, -..."
1,1,8.41,0,Movie,Original,0,0,1,0,0,...,0,0,0,0,0,0,"[-0.00041281452, 0.0009827461, -1.9511826e-05,...","[0.0014435188, -0.0047937236, -0.0007585628, 0...","[-0.003499575, -0.0029092114, 0.0023581742, -0...","[0.009648528, 0.007324829, 0.0012616563, -0.00..."
2,0,8.31,1,TV,Manga,0,0,0,0,0,...,0,0,0,0,0,0,"[0.0004804233, 0.00017798069, 0.00081434444, 0...","[-0.0012914204, 0.009018698, -0.005590031, 0.0...","[-0.0009417677, -0.0010257291, -0.0076476308, ...","[0.0081322715, -0.004457334, -0.0010683573, 0...."
3,0,7.34,0,TV,Original,0,0,0,0,0,...,0,0,0,0,0,0,"[-0.0007236848, 0.0008979221, -0.00080057426, ...","[0.0047848406, 0.00014208879, -0.001255269, -0...","[-0.0008193427, 0.00028111576, 0.0035369773, -...","[-0.0087274825, 0.0021301615, -0.0008735442, -..."
4,0,7.04,1,TV,Manga,0,0,0,0,0,...,0,0,0,0,0,0,"[-0.00085029885, 0.0021997162, 5.6877518e-05, ...","[-0.0014229012, 0.0012140521, 0.0023713761, -0...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.007879351, 0.0024533845, -0.0009934164, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15015,5,5.52,0,Music,Original,1,0,0,0,0,...,0,0,1,0,0,0,"[-0.00030113, 0.0038219758, -0.00013590511, 0....","[0.0075038527, -0.00063257816, -0.0046589146, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ..."
15016,5,5.52,0,Music,Original,0,0,0,0,0,...,0,0,1,0,0,0,"[-0.002198061, 0.0022400143, -0.0001623089, -0...","[0.009823671, 0.006244389, 0.004912238, 0.0022...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ..."
15031,3,5.29,0,Special,Original,0,0,0,0,0,...,0,0,1,0,0,0,"[-0.00095495634, 0.00057553774, 0.0023163885, ...","[0.00048050284, 0.0007103888, 0.0027244545, 0....","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ..."
15042,4,6.91,1,ONA,Manga,0,0,0,0,0,...,0,0,0,0,0,0,"[-0.00076304964, -0.001008537, -0.0010246095, ...","[-0.0016728224, 9.057795e-05, -0.0020517611, -...","[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00023748518, 0.0042191767, 0.0021141458, 0..."


In [45]:
bins = [0, 3, 6, 8, 10]
labels = ['Poor', 'Average', 'Good', 'Excellent']
df['Rating_category'] = pd.cut(df['Rating'], bins=bins, labels=labels)
df = df.drop('Rating', axis=1)

In [46]:
# replace categorical values with numerical values
df['Rating_category'] = df['Rating_category'].replace({'Poor': 0, 'Average': 1, 'Good': 2, 'Excellent': 3})

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
y = df['Rating_category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
import lightgbm as lgb

params = {
    'learning_rate': 0.1,
    'max_depth': 5,
    'min_child_samples': 30,
    'n_estimators': 200,
    'num_leaves': 30
}
clf = lgb.LGBMClassifier(**params)

In [49]:
from sklearn.ensemble import BaggingClassifier
# Create a bagging classifier using the Random Forest model
bagging_clf = BaggingClassifier(estimator=clf, n_estimators=10, random_state=42)

# Train the bagging classifier on the training data
bagging_clf.fit(X_train, y_train)

# Make predictions on the test set using the bagging classifier
y_pred = bagging_clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7684964200477327


In [50]:
joblib.dump(bagging_clf, 'bagging_clf_final.pkl')

['bagging_clf_final.pkl']

In [51]:
import lightgbm as lgb
# 'learning_rate': 0.1, 'n_estimators': 200, 'num_leaves': 30
# Create and train a LightGBM classifier
clf = lgb.LGBMClassifier(**params)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

joblib.dump(clf, 'clf_final.pkl')

Accuracy: 0.7601431980906921


['clf_final.pkl']